### Source: https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=2eSvM9zX_2d3
#### gemma-llama kernel on quadro

In [ ]:
# # Installs Unsloth, Xformers (Flash Attention) and all other packages!
# # !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# # !pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

# !pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
# # from unsloth import FastLanguageModel
# # import torch
# # max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# # dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# # load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# # fourbit_models = [
# #     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
# #     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
# #     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
# #     "unsloth/llama-3-8b-Instruct-bnb-4bit",
# #     "unsloth/llama-3-70b-bnb-4bit",
# #     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
# #     "unsloth/Phi-3-medium-4k-instruct",
# #     "unsloth/mistral-7b-bnb-4bit",
# #     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# # ] # More models at https://huggingface.co/unsloth

# # model, tokenizer = FastLanguageModel.from_pretrained(
# #     model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
# #     max_seq_length = max_seq_length,
# #     dtype = dtype,
# #     load_in_4bit = load_in_4bit,
# #     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# # )


# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
#     "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
#     "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
#     "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
# ] # More models at https://huggingface.co/unsloth


In [ ]:
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Llama-3.2-3B-Instruct",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

In [ ]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 16,
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

In [4]:
# # assuming the dataset already in instruction, input, output format

# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs       = examples["input"]
#     outputs      = examples["output"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }


# from datasets import load_dataset
# dataset = load_dataset("json", data_files= "../../Axolotl/axolotl/Comment_Generation/train.jsonl")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [5]:
# import os
# wandb_project = "unsloth-llama3.2-five-epochs-review"
# if len(wandb_project) > 0:
#     os.environ["WANDB_PROJECT"] = wandb_project


### num_train_epochs=1, and max_steps can be set None

In [6]:
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset['train'],
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     dataset_num_proc = 2,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 500,
#         # max_steps = "None",
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1000,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "qlora-llama3.2-five-epochs-review",
#         num_train_epochs=5,
#         report_to="wandb",
#         save_steps=10000,
#     ),
# )

In [ ]:
# # Show current memory stat before training
# gpu_stats = torch.cuda.get_device_properties(0)
# start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
# print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
# print(f"{start_gpu_memory} GB of memory reserved.")

In [8]:
# check wandb login, use wandb login --relogin if needed

In [ ]:
# trainer_stats = trainer.train()

In [ ]:
# # title Show final memory and time stats
# used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
# used_percentage = round(used_memory         /max_memory*100, 3)
# lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
# print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
# print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
# print(f"Peak reserved memory = {used_memory} GB.")
# print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
# print(f"Peak reserved memory % of max memory = {used_percentage} %.")
# print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
# test_file = "../../Axolotl/axolotl/Comment_Generation/test.jsonl"
# test_dataset = load_dataset("json", data_files= test_file)

In [ ]:
# test_dataset

In [ ]:
# test_dataset['train'][0]

In [ ]:
# # inference example
# # alpaca_prompt = Copied from above
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         # "Continue the fibonnaci sequence.", # instruction
#         # "1, 1, 2, 3, 5, 8", # input
#         # "", # output - leave this blank for generation!
#         test_dataset['train']['instruction'][0],
#         test_dataset['train']['input'][0],
#         "",
#     )
# ], return_tensors = "pt").to("cuda:2")

# outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True)
# tokenizer.batch_decode(outputs)

In [ ]:
# model.save_pretrained("qlora-llama3.2-five-epochs-review") # Local saving
# tokenizer.save_pretrained("qlora-llama3.2-five-epochs-review")

### Inference

In [5]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "qlora-llama3.2-five-epochs-review", # "lora_model_test", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Quadro RTX 5000. Max memory: 15.56 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`
Unsloth 2024.10.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


In [6]:
import json
# read the first 500 lines of the test data
test_data_file_path = "../../Llama/msg-test-5000-tuned.jsonl"
test_data = []
with open(test_data_file_path, 'r') as f:
    for i in range(5000): # 500
        line = f.readline()
        data = json.loads(line)
        test_data.append(data)


# write all the output part of the test data to a file as the gold for comparison
gold_file_path = "msg-5000-gold-5-epochs.txt"
with open(gold_file_path, 'w') as f:
    for data in test_data:
        # remove "New Code: " from the output
        value = data['output'].replace("New Code: ", "")
        f.write(value + "\n")

In [7]:
from datasets import load_dataset

test_file = "../../Llama/msg-test-5000-tuned.jsonl"
test_dataset = load_dataset("json", data_files= test_file)

Generating train split: 5000 examples [00:00, 241254.39 examples/s]


In [10]:
for i in range(5000):
    # inference example
# alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            # "Continue the fibonnaci sequence.", # instruction
            # "1, 1, 2, 3, 5, 8", # input
            # "", # output - leave this blank for generation!
            test_dataset['train']['instruction'][i],
            test_dataset['train']['input'][i],
            "",
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
    tokenizer.batch_decode(outputs)
    # write the output to a file
    with open("msg-5000-output-5-epochs.txt", 'a') as f:
        f.write(tokenizer.batch_decode(outputs)[0] + "\n")

### Must check with increased max_new_token!

In [7]:
# write a code that checks the output file and retrieves all lines following ### Response:\n
# write the output to a new file
output_file_path = "msg-5000-pred-5-epochs.txt"
with open("msg-5000-output-5-epochs.txt", 'r') as f:
    lines = f.readlines()
    with open(output_file_path, 'w') as f:
        for i in range(len(lines)):
            # if "New Code:" in lines[i]:
            #     # write the next line trimming <|end_of_text|> from the end
            #     f.write(lines[i][9:])
            if "### Response" in lines[i]:
                f.write(lines[i+1][:-16])
                f.write("\n")

In [ ]:
print("Done Training and Saving")